<a href="https://colab.research.google.com/github/ghasemieh/Duplicate-Bug-Identification-System/blob/master/Duplicate_BugReport_V9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import progressbar as pb
import time
import warnings
import text_processing as tp
warnings.filterwarnings('ignore')
import pickle

In [0]:
# Import dataset
data = pd.read_csv('TestData2500.csv',sep=',')
# Remove nan from the main dataset
tp.remove_nan(data)

Remove NA
Before removing the NaN:
id                 0
product            0
component          0
reporter           0
bug_status         0
resolution         0
priority           0
bug_severity       0
version            0
short_desc         1
opendate           0
dup_list        1499
root_id         1406
disc_id         1406
dtype: int64

After removing the NaN:
 id                 0
product            0
component          0
reporter           0
bug_status         0
resolution         0
priority           0
bug_severity       0
version            0
short_desc         0
opendate           0
dup_list        1498
root_id         1405
disc_id         1405
dtype: int64


In [0]:
# Preprocessing the short_desc
# processed_data_df = tp.text_preprocessing(data,"short_desc")
# # Save to file to save time
# processed_data_df.to_csv('processed_TestData2500_df.csv',index=False)

N/A% (0 of 1594) |                       | Elapsed Time: 0:00:00 ETA:  --:--:--

Preprocessing the text


 99% (1592 of 1594) |################### | Elapsed Time: 0:00:42 ETA:   0:00:00

Remove NA
Before removing the NaN:
id                         0
short_desc_processed       0
product                    0
component                  0
reporter                   0
bug_status                 0
resolution                 0
priority                   0
bug_severity               0
version                    0
short_desc                 0
opendate                   0
dup_list                1498
root_id                 1405
disc_id                 1405
dtype: int64

After removing the NaN:
 id                         0
short_desc_processed       0
product                    0
component                  0
reporter                   0
bug_status                 0
resolution                 0
priority                   0
bug_severity               0
version                    0
short_desc                 0
opendate                   0
dup_list                1498
root_id                 1405
disc_id                 1405
dtype: int64
Text preprocessing --- 42.19237446784973 se

In [0]:
# Load prosecced df from file to sasve time
processed_data_df = pd.read_csv('processed_TestData2500_df.csv',sep=',')

In [0]:
# Model-1: Similarity Score - Word2vec -------------------------------------------------------------------------
import spacy
nlp = spacy.load('en_core_web_lg')
execution_count_word2vec = 0
processed_data_nlp_df = []

# Convert short_desc str to nlp format to fasten the computation process 
def word2vec_preprocess(df):
    print('Convert short_desc str to nlp format')
    sample_size = len(df)
    progress = pb.ProgressBar(maxval = sample_size).start()
    progvar  = 0
    processed_data_nlp = []
    for tup in df.itertuples():
        processed_data_nlp.append((tup.id,tup.product,nlp(tup.short_desc_processed))) 
        progress.update(progvar + 1)
        progvar += 1
    global processed_data_nlp_df
    processed_data_nlp_df = pd.DataFrame(processed_data_nlp, columns=['id','product','short_desc_processed'])
    global execution_count_word2vec
    execution_count_word2vec += 1

# Calculate the cosine similarity score
def word2vec_similarity(id,df):
    if execution_count_word2vec == 0:
        word2vec_preprocess(df)
    similarities_score_list = []
    product_main = processed_data_nlp_df.loc[lambda df: df['id'] == id,'product'].array[0]
    short_desc_processed_main = processed_data_nlp_df.loc[lambda df: df['id'] == id,'short_desc_processed'].array[0]
    for doc in processed_data_nlp_df.itertuples():
        product_other = processed_data_nlp_df.loc[lambda df: df['id'] == doc.id,'product'].array[0]
        if product_main == product_other:
            similarity_score = doc.short_desc_processed.similarity(short_desc_processed_main)
            similarities_score_list.append((doc.id,similarity_score))
    #convert to dataframe
    word2vec_similarities_score_df = pd.DataFrame(similarities_score_list, columns=['id','word2vec_score'] )
    word2vec_similarities_score_df = word2vec_similarities_score_df.reset_index(drop=True)
    return word2vec_similarities_score_df

In [0]:
# Model-2: Similarity Score - TF-idf ----------------------------------------------------------------------------
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

tfidf_cosine_similarities = []
execution_count_tfidf = 0

def tfidf_preprocess(df):
    X_train = df['short_desc_processed']
    print('TF-idf Vectorization and similarity score computation')
    # Vectorization
    vectorizer = TfidfVectorizer()
    tfidf = vectorizer.fit_transform(X_train)
    # Calculate the cosine similarity score
    global tfidf_cosine_similarities
    tfidf_cosine_similarities = linear_kernel(tfidf)
    global shape_tfidf
    shape_tfidf = tfidf_cosine_similarities.shape[0]
    print('TF-idf preprocess done')
    global execution_count_tfidf
    execution_count_tfidf += 1

def tfidf_similarities(id,df):
    if execution_count_tfidf == 0:
        tfidf_preprocess(df)
    index_main   = df.loc[lambda df: df['id'] == id].index.array[0]
    product_main = df.loc[lambda df: df['id'] == id,'product'].array[0]
    tfidf_cosine_similarities_list = []
    for index_other in range(shape_tfidf):
        id_other      = df.iloc[index_other]['id']
        product_other = df.iloc[index_other]['product']
        if product_main == product_other:
            tfidf_cosine_similarities_list.append([id_other,tfidf_cosine_similarities[index_main,index_other]])
    #Conver to dataframe
    tfidf_cosine_similarities_score_df = pd.DataFrame(tfidf_cosine_similarities_list, columns=['id','tfidf_score'])
    tfidf_cosine_similarities_score_df = tfidf_cosine_similarities_score_df.reset_index(drop=True)
    return tfidf_cosine_similarities_score_df

In [0]:
# Model-3: Similarity Score - BM25F -----------------------------------------------------------------------------
from rank_bm25 import BM25Okapi

processed_corpus_list = []
bm25 = []
execution_count_bm25 = 0

# preprocess - tokenize the short_desc to token
def bm25_preprocess(df):
    print('preprocess - tokenize the short_desc to token')
    sample_size = len(df)
    global processed_corpus_list
    processed_corpus_list = []
    for x in df.itertuples():
        short_desc_splited = x.short_desc_processed.split(" ")
        processed_corpus_list.append(short_desc_splited)
    # Create a MB24 Object with the corpus
    global bm25
    bm25 = BM25Okapi(processed_corpus_list)
    global execution_count_bm25
    execution_count_bm25 += 1
    
# Calculate the similarity score
def bm25_similarity(id,df):
    if execution_count_bm25 == 0:
        bm25_preprocess(df)
    index_main    = df.loc[lambda df: df['id'] == id].index.array[0]
    product_main  = df.loc[lambda df: df['id'] == id,'product'].array[0]
    query         = processed_corpus_list[index_main]
    doc_scores    = bm25.get_scores(query)
    doc_scores_df = pd.DataFrame(doc_scores, columns=['bm25_score'])
    # add id to the score list and remove unsimiliar product
    blanks = []
    for x in doc_scores_df.itertuples():
        id_other      = df.iloc[x.Index]['id']
        product_other = df.iloc[x.Index]['product']
        # add id to the score list
        doc_scores_df.loc[x.Index,'id'] = id_other
        if product_main != product_other:
            blanks.append(x.Index)    
    doc_scores_df.drop(blanks,inplace=True)   
    doc_scores_df = doc_scores_df.reset_index(drop = True)  
    return doc_scores_df

# Assessment the Model Accuracy

In [0]:
# Assessment the Model Accuracy -------------------------------------------------------------------------------
# duplicate_df = pd.read_csv('https://zenodo.org/record/400614/files/apache.relations.csv?download=1',sep=',')
duplicate_df = pd.read_csv('duplicate_df.csv',sep=',')

In [0]:
# Calculate the similarity scores and return the first n top scoes
def similarity_score(id, df,top_n):
    word2vec_similarity_df = word2vec_similarity(id, df).sort_values(by=['word2vec_score'],ascending=False).head(top_n)
    tfidf_similarity_df    = tfidf_similarities(id, df).sort_values(by=['tfidf_score'],ascending=False).head(top_n)
    bm25_similarity_df     = bm25_similarity(id, df).sort_values(by=['bm25_score'],ascending=False).head(top_n)    
    return word2vec_similarity_df, tfidf_similarity_df, bm25_similarity_df

In [0]:
# Calculate the similarity score for the reported duplicated bug report
start_time  = time.time()
sample_size = len(duplicate_df)
progress    = pb.ProgressBar(maxval = sample_size).start()
progvar     = 0
duplicated_similarity_score_list = []

for tup in duplicate_df.itertuples():
    word2vec_similarity_df, tfidf_similarity_df, bm25_similarity_df = similarity_score(tup.id, processed_data_df,20)
    duplicated_similarity_score_list.append([tup.id,tup.dup,word2vec_similarity_df,tfidf_similarity_df,bm25_similarity_df])
    progress.update(progvar + 1)
    progvar += 1
    
print("-Total- %s seconds ---" % (time.time() - start_time))

100% (83 of 83) |########################| Elapsed Time: 0:03:44 ETA:  00:00:00

-Total- 224.28654527664185 seconds ---


In [0]:
# Save the list to a file since it takes 8 hours to create it
with open("duplicated_similarity_score_list_2500.txt", "wb") as fp:   #Pickling
    pickle.dump(duplicated_similarity_score_list, fp)

# # Read the list from a file 
with open("duplicated_similarity_score_list_2500.txt", "rb") as fp:   # Unpickling
    duplicated_similarity_score_list = pickle.load(fp)

## Calculate the Recall Rate

In [0]:
# Calculate the Recall rate
def recall_rate_calculation(name_of_algorthem):
    if name_of_algorthem == 'word2vec':
        select_algorithm = 2
        print('word2vec result:')
    elif name_of_algorthem == 'tfidf':
        select_algorithm = 3
        print('TF-idf result:')
    elif name_of_algorthem == 'bm25':
        select_algorithm = 4
        print('bm25 result:')
    else:
        return "Wrong selection"
    found_counter = 0
    not_found_counter = 0
    for x in duplicated_similarity_score_list:
        dup = x[1]
        df = x[select_algorithm]
        if df.loc[df['id'] == dup].empty:
            not_found_counter +=1
        else:
            found_counter +=1
    print('Num of duplicated report found: ',found_counter)
    print('Num of duplicated report not found: ',not_found_counter)
    print('Recall (TP/TP+FN): ', round(found_counter/len(duplicated_similarity_score_list)*100,2),'%\n')

In [0]:
# Check the results
recall_rate_calculation('word2vec')
recall_rate_calculation('tfidf')
recall_rate_calculation('bm25')

word2vec result:
Num of duplicated report found:  48
Num of duplicated report not found:  35
Recall (TP/TP+FN):  57.83 %

TF-idf result:
Num of duplicated report found:  51
Num of duplicated report not found:  32
Recall (TP/TP+FN):  61.45 %

bm25 result:
Num of duplicated report found:  50
Num of duplicated report not found:  33
Recall (TP/TP+FN):  60.24 %



## Calculate the MRR 
![image.png](attachment:image.png)

In [0]:
# Calculate the MRR
def MRR_rate_calculation(name_of_algorthem):
    if name_of_algorthem == 'word2vec':
        select_algorithm = 2
        print('word2vec result:')
    elif name_of_algorthem == 'tfidf':
        select_algorithm = 3
        print('TF-idf result:')
    elif name_of_algorthem == 'bm25':
        select_algorithm = 4
        print('bm25 result:')
    else:
        return "Wrong selection"
    found_counter = 0
    not_found_counter = 0
    revers_sum = 0
    for x in duplicated_similarity_score_list:
        dup = x[1]
        df = x[select_algorithm].reset_index(drop=True)
        if df.loc[df['id'] == dup].empty:
            not_found_counter +=1
        else:
            found_counter +=1
            for tup in df.itertuples():
                if int(tup.id) == dup:
                    if tup.Index ==0 :
                        revers_sum += 1/(tup.Index+1)
                    else:
                        revers_sum += 1/(tup.Index)
                
    print('Num of duplicated report found: ',found_counter)
    print('Num of duplicated report not found: ',not_found_counter)
    print('MRR_rate: ', round((1/len(duplicated_similarity_score_list))*revers_sum,5),'\n')

In [0]:
MRR_rate_calculation('word2vec')
MRR_rate_calculation('tfidf')
MRR_rate_calculation('bm25')

word2vec result:
Num of duplicated report found:  48
Num of duplicated report not found:  35
MRR_rate:  0.3138 

TF-idf result:
Num of duplicated report found:  51
Num of duplicated report not found:  32
MRR_rate:  0.35534 

bm25 result:
Num of duplicated report found:  50
Num of duplicated report not found:  33
MRR_rate:  0.34919 



## Calculate the MAP
![image.png](attachment:image.png)

In [0]:
# Calculate the MAP
def MAP_rate_calculation(name_of_algorthem, threshold):
    if name_of_algorthem == 'word2vec':
        select_algorithm = 2
        print('word2vec result:')
    elif name_of_algorthem == 'tfidf':
        select_algorithm = 3
        print('TF-idf result:')
    elif name_of_algorthem == 'bm25':
        select_algorithm = 4
        print('bm25 result:')
    else:
        return "Wrong selection"
    found_counter = 0
    not_found_counter = 0
    sum_overall_AP = 0
    overall_AP = 0
    for x in duplicated_similarity_score_list:
        dup = x[1]
        df = x[select_algorithm].reset_index(drop=True)
        
        if df.loc[df['id'] == dup].empty:
            not_found_counter +=1
        else:
            num_true_positive = 1
            sumAP = 0
            for tup in df.itertuples():
                if int(tup.id) == dup:
                    found_counter +=1
                    if tup.Index ==0 :
                        sumAP += num_true_positive/(tup.Index+1)
                    else:
                        sumAP += num_true_positive/(tup.Index)
                num_true_positive += 1 
            overall_AP = (1/num_true_positive) * sumAP
        sum_overall_AP += overall_AP
                
    print('Num of duplicated report found: ',found_counter)
    print('Num of duplicated report not found: ',not_found_counter)
    print('MAP_rate: ',round(sum_overall_AP/len(duplicated_similarity_score_list)*100,5), '\n')

In [0]:
MAP_rate_calculation('word2vec',0.5)
MAP_rate_calculation('tfidf',0.3)
MAP_rate_calculation('bm25',0.5)

word2vec result:
Num of duplicated report found:  48
Num of duplicated report not found:  35
MAP_rate:  6.30397 

TF-idf result:
Num of duplicated report found:  51
Num of duplicated report not found:  32
MAP_rate:  6.84587 

bm25 result:
Num of duplicated report found:  50
Num of duplicated report not found:  33
MAP_rate:  6.70286 



## TF-idf PySpark

### Feature Transformations

In [0]:
from pyspark.sql import SparkSession

In [0]:
spark = SparkSession.builder.appName('nlp').getOrCreate()
data = spark.read.csv("processed_TestData2500_df.csv",inferSchema=True,header=True,sep=',')

In [0]:
from pyspark.ml.feature import Tokenizer,StopWordsRemover, CountVectorizer,IDF,StringIndexer

tokenizer = Tokenizer(inputCol="short_desc_processed", outputCol="token_short_desc_processed")
count_vec = CountVectorizer(inputCol='token_short_desc_processed',outputCol='c_vec')
idf = IDF(inputCol="c_vec", outputCol="tf_idf")

### Pipeline

In [0]:
from pyspark.ml import Pipeline

In [0]:
data_prep_pipe = Pipeline(stages=[tokenizer,count_vec,idf])
cleaner = data_prep_pipe.fit(data)
clean_data = cleaner.transform(data)
clean_data = clean_data.select(['id','product','tf_idf'])
tfidf = clean_data.select(['tf_idf'])

In [0]:
clean_data.show()

+---+--------+--------------------+
| id| product|              tf_idf|
+---+--------+--------------------+
|  2| Log4j -|(1590,[49,404,572...|
|  3| Log4j -|(1590,[4,738],[2....|
| 22|Apache h|(1590,[3,39,62,27...|
| 27| Log4j -|(1590,[3,190,199]...|
| 29| Log4j -|(1590,[3,59,92,11...|
| 31| Log4j -|(1590,[44,49,123,...|
| 32| Log4j -|(1590,[424],[5.76...|
| 34| Log4j -|(1590,[8,95,112,6...|
| 35| Log4j -|(1590,[121,127,47...|
| 43| Log4j -|(1590,[0,193,395,...|
| 44| Log4j -|(1590,[3,156],[2....|
| 45| Batik -|(1590,[30,34,45,2...|
| 46| Batik -|(1590,[30,34,618]...|
| 47| Batik -|(1590,[292,618,88...|
| 48| Batik -|(1590,[0,452,839,...|
| 49| Batik -|(1590,[619,1124,1...|
| 50| Batik -|(1590,[74,190,195...|
| 51| Batik -|(1590,[35,212,541...|
| 52| Batik -|(1590,[93,225,312...|
| 53| Batik -|(1590,[116,219,61...|
+---+--------+--------------------+
only showing top 20 rows



In [0]:
# X_train = data.collect()
# vectorizer = TfidfVectorizer()
# tfidf = vectorizer.fit_transform(X_train)
# # tfidf_cosine_similarities = linear_kernel(tfidf)
# # shape_tfidf = tfidf_cosine_similarities.shape[0]

In [0]:
# Model-2: Similarity Score - TF-idf ----------------------------------------------------------------------------
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

tfidf_cosine_similarities = []
execution_count_tfidf = 0

def tfidf_preprocess_pyspark(id,df):
    X_train = df['short_desc_processed']
    print('TF-idf Vectorization and similarity score computation')
    # Vectorization
    vectorizer = TfidfVectorizer()
    tfidf = vectorizer.fit_transform(X_train)
    # Calculate the cosine similarity score
    tfidf_cosine_similarities = linear_kernel(tfidf)
    print('TF-idf preprocess done')
    
    index_main   = df.loc[lambda df: df['id'] == id].index.array[0]
    product_main = df.loc[lambda df: df['id'] == id,'product'].array[0]
    tfidf_cosine_similarities_list = []
    for index_other in range(shape_tfidf):
        id_other      = df.iloc[index_other]['id']
        product_other = df.iloc[index_other]['product']
        if product_main == product_other:
            tfidf_cosine_similarities_list.append([id_other,tfidf_cosine_similarities[index_main,index_other]])
    #Conver to dataframe
    tfidf_cosine_similarities_score_df = pd.DataFrame(tfidf_cosine_similarities_list, columns=['id','tfidf_score'])
    tfidf_cosine_similarities_score_df = tfidf_cosine_similarities_score_df.reset_index(drop=True)
    return tfidf_cosine_similarities_score_df

In [0]:
from pyspark.sql.functions import udf, struct, col
from pyspark.sql.types import * 
import pyspark.sql.functions as func

In [0]:
schema = StructType([
    StructField("id", IntegerType(), False),
    StructField("tfidf_score", IntegerType(), False)
])

In [0]:
tfidf_preprocess_udf = udf(lambda z: tfidf_preprocess_pyspark(id, df), schema)

In [0]:
df_new = df.select('id', tfidf_preprocess_udf(2,processed_data_df))
df_new.show()

Traceback (most recent call last):
  File "/home/alireza/anaconda3/lib/python3.7/site-packages/pyspark/serializers.py", line 590, in dumps
    return cloudpickle.dumps(obj, 2)
  File "/home/alireza/anaconda3/lib/python3.7/site-packages/pyspark/cloudpickle.py", line 863, in dumps
    cp.dump(obj)
  File "/home/alireza/anaconda3/lib/python3.7/site-packages/pyspark/cloudpickle.py", line 260, in dump
    return Pickler.dump(self, obj)
  File "/home/alireza/anaconda3/lib/python3.7/pickle.py", line 437, in dump
    self.save(obj)
  File "/home/alireza/anaconda3/lib/python3.7/pickle.py", line 504, in save
    f(self, obj) # Call unbound method with explicit self
  File "/home/alireza/anaconda3/lib/python3.7/pickle.py", line 774, in save_tuple
    save(element)
  File "/home/alireza/anaconda3/lib/python3.7/pickle.py", line 504, in save
    f(self, obj) # Call unbound method with explicit self
  File "/home/alireza/anaconda3/lib/python3.7/site-packages/pyspark/cloudpickle.py", line 400, in save

PicklingError: Could not serialize object: Py4JError: An error occurred while calling o352.__getstate__. Trace:
py4j.Py4JException: Method __getstate__([]) does not exist
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:318)
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:326)
	at py4j.Gateway.invoke(Gateway.java:274)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:238)
	at java.lang.Thread.run(Thread.java:748)

